<a href="https://colab.research.google.com/github/ParkSomin23/Voice_EmotionDetection/blob/master/EMO_1_Data_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import librosa
import librosa.display
import IPython.display as ipd

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram

import pandas as pd
import glob
from sklearn.metrics import confusion_matrix

import os
import sys

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
RAVDESS = "/gdrive/My Drive/RAVDESS/audio_speech_actors_01-24/"
SAVEE = "/gdrive/My Drive/SAVEE/"
CREMA = "/gdrive/My Drive/CREMA-D/"
TESS = "/gdrive/My Drive/TESS/TESS Toronto emotional speech set data/"

In [ ]:
filename = os.listdir(RAVDESS)
filename

['Actor_09',
 'Actor_11',
 'Actor_23',
 'Actor_02',
 'Actor_18',
 'Actor_22',
 'Actor_19',
 'Actor_08',
 'Actor_14',
 'Actor_04',
 'Actor_21',
 'Actor_13',
 'Actor_01',
 'Actor_05',
 'Actor_07',
 'Actor_15',
 'Actor_12',
 'Actor_16',
 'Actor_03',
 'Actor_10',
 'Actor_24',
 'Actor_06',
 'Actor_20',
 'Actor_17']

# RAVDESS DATASET

In [ ]:
dir_list = os.listdir(RAVDESS)

emotion = []
gender = []
path = []

for i in dir_list:                              # i : Actor_01, Actor_02 ...
  file_name  = os.listdir(RAVDESS + i)
  for f in file_name:
    part = f.split('.')[0].split('-')
    emotion.append(int(part[2]))
    gen = int(part[6])
    if gen % 2 == 0:
      gen = "female"
    else:
      gen = "male"
    gender.append(gen)
    path.append(RAVDESS + i + '/' + f)

RAVDESS_df = pd.DataFrame(emotion)
RAVDESS_df = RAVDESS_df.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
RAVDESS_df = pd.concat([pd.DataFrame(gender), RAVDESS_df], axis = 1)

RAVDESS_df.columns = ['gender', 'emotion']
RAVDESS_df['labels'] =RAVDESS_df.gender + '_' + RAVDESS_df.emotion
RAVDESS_df['source'] = 'RAVDESS' 

RAVDESS_df = pd.concat([RAVDESS_df, pd.DataFrame(path, columns=['path'])], axis = 1)
RAVDESS_df = RAVDESS_df.drop(['gender', 'emotion'], axis = 1)
RAVDESS_df.labels.value_counts()

female_neutral     144
male_neutral       144
male_happy          96
female_surprise     96
female_happy        96
male_angry          96
female_fear         96
female_disgust      96
female_sad          96
female_angry        96
male_sad            96
male_fear           96
male_surprise       96
male_disgust        96
Name: labels, dtype: int64

In [ ]:
RAVDESS_df

,labels,source,path
0,male_neutral,RAVDESS,/gdrive/My Drive/RAVDESS/audio_speech_actors_0...
1,male_neutral,RAVDESS,/gdrive/My Drive/RAVDESS/audio_speech_actors_0...
2,male_neutral,RAVDESS,/gdrive/My Drive/RAVDESS/audio_speech_actors_0...
3,male_sad,RAVDESS,/gdrive/My Drive/RAVDESS/audio_speech_actors_0...
4,male_angry,RAVDESS,/gdrive/My Drive/RAVDESS/audio_speech_actors_0...
...,...,...,...
1435,male_surprise,RAVDESS,/gdrive/My Drive/RAVDESS/audio_speech_actors_0...
1436,male_disgust,RAVDESS,/gdrive/My Drive/RAVDESS/audio_speech_actors_0...
1437,male_disgust,RAVDESS,/gdrive/My Drive/RAVDESS/audio_speech_actors_0...
1438,male_disgust,RAVDESS,/gdrive/My Drive/RAVDESS/audio_speech_actors_0...


# SAVEE DATASET

all male dataset

In [ ]:
dir_list

In [ ]:
dir_list = os.listdir(SAVEE)

emotion = []
path = []

for i in dir_list:
    if i[-8:-6]=='_a':
        emotion.append('male_angry')
    elif i[-8:-6]=='_d':
        emotion.append('male_disgust')
    elif i[-8:-6]=='_f':
        emotion.append('male_fear')
    elif i[-8:-6]=='_h':
        emotion.append('male_happy')
    elif i[-8:-6]=='_n':
        emotion.append('male_neutral')
    elif i[-8:-6]=='sa':
        emotion.append('male_sad')
    elif i[-8:-6]=='su':
        emotion.append('male_surprise')
    else:
        emotion.append('male_error')
    
    path.append(SAVEE + i)

SAVEE_df = pd.DataFrame(emotion, columns = ['labels'])
SAVEE_df['source'] = 'SAVEE'
SAVEE_df = pd.concat([SAVEE_df, pd.DataFrame(path, columns=['path'])], axis=1)
SAVEE_df.labels.value_counts()

male_neutral     60
male_angry       45
male_disgust     45
male_fear        44
male_surprise    30
male_sad         30
male_happy       30
Name: labels, dtype: int64

In [ ]:
SAVEE_df

,labels,source,path
0,male_neutral,SAVEE,/gdrive/My Drive/SAVEE/DC_n11.wav
1,male_angry,SAVEE,/gdrive/My Drive/SAVEE/DC_a12.wav
2,male_fear,SAVEE,/gdrive/My Drive/SAVEE/JE_f04.wav
3,male_disgust,SAVEE,/gdrive/My Drive/SAVEE/JK_d15.wav
4,male_happy,SAVEE,/gdrive/My Drive/SAVEE/DC_h14.wav
...,...,...,...
279,male_disgust,SAVEE,/gdrive/My Drive/SAVEE/DC_d12.wav
280,male_fear,SAVEE,/gdrive/My Drive/SAVEE/JK_f02.wav
281,male_angry,SAVEE,/gdrive/My Drive/SAVEE/DC_a09.wav
282,male_disgust,SAVEE,/gdrive/My Drive/SAVEE/JE_d05.wav


# TESS DATASET

all female

In [ ]:
dir_list = os.listdir(TESS)
dir_list.sort()
dir_list

['OAF_Fear',
 'OAF_Pleasant_surprise',
 'OAF_Sad',
 'OAF_angry',
 'OAF_disgust',
 'OAF_happy',
 'OAF_neutral',
 'YAF_angry',
 'YAF_disgust',
 'YAF_fear',
 'YAF_happy',
 'YAF_neutral',
 'YAF_pleasant_surprised',
 'YAF_sad']

In [ ]:
path = []
emotion = []

for i in dir_list:
  file_name = os.listdir(TESS + i)
  for f in file_name:
    if i == 'OAF_angry' or i == 'YAF_angry':
      emotion.append('female_angry')
    elif i == 'OAF_disgust' or i == 'YAF_disgust':
      emotion.append('female_disgust')
    elif i == 'OAF_Fear' or i == 'YAF_fear':
      emotion.append('female_fear')
    elif i == 'OAF_happy' or i == 'YAF_happy':
      emotion.append('female_happy')
    elif i == 'OAF_neutral' or i == 'YAF_neutral':
      emotion.append('female_neutral')                                
    elif i == 'OAF_Pleasant_surprise' or i == 'YAF_pleasant_surprised':
      emotion.append('female_surprise')               
    elif i == 'OAF_Sad' or i == 'YAF_sad':
      emotion.append('female_sad')
    else:
      emotion.append('Unknown')
    path.append(TESS + i + "/" + f)

TESS_df = pd.DataFrame(emotion, columns=['labels'])
TESS_df['source'] = 'TESS'
TESS_df = pd.concat([TESS_df, pd.DataFrame(path, columns=['path'])], axis=1)
TESS_df.labels.value_counts()

female_fear        400
female_sad         400
female_neutral     400
female_surprise    400
female_happy       400
female_disgust     400
female_angry       400
Name: labels, dtype: int64

In [ ]:
TESS_df

,labels,source,path
0,female_fear,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
1,female_fear,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
2,female_fear,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
3,female_fear,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
4,female_fear,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
...,...,...,...
2795,female_sad,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
2796,female_sad,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
2797,female_sad,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
2798,female_sad,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...


#CREMA-D

In [ ]:
dir_list = os.listdir(CREMA)
dir_list.sort()
dir_list[:10]

['1001_DFA_ANG_XX.wav',
 '1001_DFA_DIS_XX.wav',
 '1001_DFA_FEA_XX.wav',
 '1001_DFA_HAP_XX.wav',
 '1001_DFA_NEU_XX.wav',
 '1001_DFA_SAD_XX.wav',
 '1001_IEO_ANG_HI.wav',
 '1001_IEO_ANG_LO.wav',
 '1001_IEO_ANG_MD.wav',
 '1001_IEO_DIS_HI.wav']

In [ ]:
gender = []
emotion = []
path = []
female = [1002,1003,1004,1006,1007,1008,1009,1010,1012,1013,1018,1020,1021,1024,1025,1028,1029,1030,1037,1043,1046,1047,1049,
          1052,1053,1054,1055,1056,1058,1060,1061,1063,1072,1073,1074,1075,1076,1078,1079,1082,1084,1089,1091]

for i in dir_list:
  part = i.split('_')
  
  if int(part[0]) in female:
    gen = "female"
  else:
    gen = "male"
  gender.append(gen)

  if part[2] == 'SAD':
    emotion.append(gen + '_sad')
  elif part[2] == 'ANG':
    emotion.append(gen + '_angry')
  elif part[2] == 'DIS':
    emotion.append(gen + '_disgust')
  elif part[2] == 'FEA':
    emotion.append(gen + '_fear')
  elif part[2] == 'HAP':
    emotion.append(gen + '_happy')
  elif part[2] == 'NEU':
    emotion.append(gen + '_neutral')
  else:
    emotion.append('Unknown')

  path.append(CREMA + i)

CREMA_df = pd.DataFrame(emotion, columns=['labels'])
CREMA_df['source'] = "CREMA"
CREMA_df = pd.concat([CREMA_df, pd.DataFrame(path, columns=['path'])], axis=1)
CREMA_df.labels.value_counts()

male_angry        635
male_disgust      635
male_fear         634
male_sad          634
male_happy        634
female_happy      573
female_fear       572
female_angry      572
female_disgust    572
female_sad        572
male_neutral      542
female_neutral    488
Name: labels, dtype: int64

In [ ]:
CREMA_df

,labels,source,path
0,male_angry,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_ANG_XX.wav
1,male_disgust,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_DIS_XX.wav
2,male_fear,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_FEA_XX.wav
3,male_happy,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_HAP_XX.wav
4,male_neutral,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_NEU_XX.wav
...,...,...,...
7058,male_happy,CREMA,/gdrive/My Drive/CREMA-D/1087_IOM_HAP_XX.wav
7059,male_neutral,CREMA,/gdrive/My Drive/CREMA-D/1087_IOM_NEU_XX.wav
7060,male_sad,CREMA,/gdrive/My Drive/CREMA-D/1087_IOM_SAD_XX.wav
7061,male_angry,CREMA,/gdrive/My Drive/CREMA-D/1087_ITH_ANG_XX.wav


In [ ]:
c_data.to_csv("dataset_crema_2.csv", mode='w')

# ALL DATA

In [ ]:
df = pd.concat([SAVEE_df, RAVDESS_df, TESS_df, CREMA_df], axis = 0)
print(df.labels.value_counts())
df.head()
df.to_csv("Data_path.csv",index=False)

female_happy       1069
female_disgust     1068
female_angry       1068
female_sad         1068
female_fear        1068
female_neutral     1032
male_disgust        776
male_angry          776
male_fear           774
male_happy          760
male_sad            760
male_neutral        746
female_surprise     496
male_surprise       126
Name: labels, dtype: int64


In [ ]:
df

,labels,source,path
0,male_neutral,SAVEE,/gdrive/My Drive/SAVEE/DC_n11.wav
1,male_angry,SAVEE,/gdrive/My Drive/SAVEE/DC_a12.wav
2,male_fear,SAVEE,/gdrive/My Drive/SAVEE/JE_f04.wav
3,male_disgust,SAVEE,/gdrive/My Drive/SAVEE/JK_d15.wav
4,male_happy,SAVEE,/gdrive/My Drive/SAVEE/DC_h14.wav
...,...,...,...
7058,male_happy,CREMA,/gdrive/My Drive/CREMA-D/1087_IOM_HAP_XX.wav
7059,male_neutral,CREMA,/gdrive/My Drive/CREMA-D/1087_IOM_NEU_XX.wav
7060,male_sad,CREMA,/gdrive/My Drive/CREMA-D/1087_IOM_SAD_XX.wav
7061,male_angry,CREMA,/gdrive/My Drive/CREMA-D/1087_ITH_ANG_XX.wav


# Separate

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
import pandas as pd
ref = pd.read_csv("/gdrive/My Drive/Data_path.csv")

In [ ]:
crema_idx = ref[ref['source']=="CREMA"].index

data = ref(crema_idx)
data

,labels,source,path
0,male_neutral,SAVEE,/gdrive/My Drive/SAVEE/DC_n11.wav
1,male_angry,SAVEE,/gdrive/My Drive/SAVEE/DC_a12.wav
2,male_fear,SAVEE,/gdrive/My Drive/SAVEE/JE_f04.wav
3,male_disgust,SAVEE,/gdrive/My Drive/SAVEE/JK_d15.wav
4,male_happy,SAVEE,/gdrive/My Drive/SAVEE/DC_h14.wav
...,...,...,...
4519,female_sad,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
4520,female_sad,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
4521,female_sad,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
4522,female_sad,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...


In [ ]:
data.to_csv("dataset_3.csv", mode='w')

In [ ]:
crema_idx = ref[ref['source']!="CREMA"].index

c_data = ref.drop(crema_idx)
c_data

,labels,source,path
4524,male_angry,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_ANG_XX.wav
4525,male_disgust,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_DIS_XX.wav
4526,male_fear,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_FEA_XX.wav
4527,male_happy,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_HAP_XX.wav
4528,male_neutral,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_NEU_XX.wav
...,...,...,...
11582,male_happy,CREMA,/gdrive/My Drive/CREMA-D/1087_IOM_HAP_XX.wav
11583,male_neutral,CREMA,/gdrive/My Drive/CREMA-D/1087_IOM_NEU_XX.wav
11584,male_sad,CREMA,/gdrive/My Drive/CREMA-D/1087_IOM_SAD_XX.wav
11585,male_angry,CREMA,/gdrive/My Drive/CREMA-D/1087_ITH_ANG_XX.wav


In [ ]:
c_data.to_csv("dataset_crema.csv", mode='w')